In [1]:
from IPython.display import clear_output
import json
import random


# ===== LOAD FILES =====
with open("Moves.json", "r") as f:
    MOVESET = json.load(f)

with open("Types.json", "r") as f:
    TYPES = json.load(f)

In [2]:

# ===== SELECTED MOVESET =====
moveset = {
    1: MOVESET["Small Heal"],
    2: MOVESET["Attack"],
    3: None,
    4: None
}

In [3]:

# GAME START AND INFO

def state_start(data):
    is_first = random.choice([True, False])
    p_type = TYPES[str(random.choice([0, 2]))]
    e_type = TYPES[str(random.choice([0, 2]))]

    data.update({
        "is_player_first": is_first,
        "player_hp": p_type["base_hp"],
        "player_type": p_type,
        "enemy_hp": e_type["base_hp"],
        "enemy_type": e_type,
        "last_actor": None
    })

    print("=== BATTLE STARTING ===")
    print(("PLAYER" if is_first else "ENEMY"), "goes first!\n")

    return "DISPLAY_INFO"


def state_display_info(data):
    print("--PLAYER INFO--")
    print("Health:", data["player_hp"])
    print("Type:", data["player_type"]["name"])
    print("\n--ENEMY INFO--")
    print("Health:", data["enemy_hp"])
    print("Type:", data["enemy_type"]["name"])
    print()

    return "PLAYER_TURN" if data["is_player_first"] else "ENEMY_TURN"

In [4]:
# PLAYER TURN

def state_player_turn(data):

    move_index = int(input("Select a move (1-4): "))
    print("=== PLAYER TURN ===")
    move = moveset.get(move_index)
    if move is None:
        print("Invalid move! Turn wasted.")
        return "ENEMY_TURN"
    
    print("YOU used:", move["type"])

    if move["type"] == "HEAL":
        data["player_hp"] += move["base_hp"]
    elif move["type"] == "ATTACK":
        data["enemy_hp"] -= move["base_hp"]

    data['last_actor'] = 'player'
    return "CHECK_WIN"

In [5]:
# ENEMY TURN
def state_enemy_turn(data):
    print("=== ENEMY TURN ===")

    e_hp = data["enemy_hp"]
    e_type = data["enemy_type"]

    if e_hp < e_type["base_hp"] * e_type["heal_threshold"]:
        if random.random() < e_type["heal_probability"]:
            data["enemy_hp"] += e_type["heal_base_hp"]
            print("ENEMY used: HEAL")

            data["last_actor"] = "enemy"
            return "CHECK_WIN"

    # Otherwise attack
    data["player_hp"] -= e_type["attack_base_hp"]
    print("ENEMY used: ATTACK")

    data["last_actor"] = "enemy"
    return "CHECK_WIN"

In [6]:
# WIN DETECTION

def state_check_win(data):
    hp = data["player_hp"]
    if hp < 0: hp = 0

    ehp = data["enemy_hp"]
    if ehp < 0: ehp = 0

    print(f"Player HP: {hp}")
    print(f"Enemy  HP: {ehp}\n")

    if hp <= 0:
        print("You lose.")
        return "END"
    if ehp <= 0:
        print("You win!")
        return "END"

    # Continue cycle
    return "ENEMY_TURN" if data['last_actor'] == "player" else "PLAYER_TURN"

In [7]:
# END STATE

def state_end(data):
    print("=== BATTLE FINISHED ===")
    return None

In [8]:
# GAME BEGIN

STATES = {
    "START": state_start,
    "DISPLAY_INFO": state_display_info,
    "PLAYER_TURN": state_player_turn,
    "ENEMY_TURN": state_enemy_turn,
    "CHECK_WIN": state_check_win,
    "END": state_end,
}


def run_fsm():
    data = {}
    state = "START"

    while state is not None:
        state = STATES[state](data)
        

In [9]:
# Start game
run_fsm()

=== BATTLE STARTING ===
ENEMY goes first!

--PLAYER INFO--
Health: 130
Type: Grass

--ENEMY INFO--
Health: 130
Type: Grass

=== ENEMY TURN ===
ENEMY used: ATTACK
Player HP: 105
Enemy  HP: 130

=== PLAYER TURN ===
YOU used: ATTACK
Player HP: 105
Enemy  HP: 120

=== ENEMY TURN ===
ENEMY used: ATTACK
Player HP: 80
Enemy  HP: 120

=== PLAYER TURN ===
YOU used: ATTACK
Player HP: 80
Enemy  HP: 110

=== ENEMY TURN ===
ENEMY used: ATTACK
Player HP: 55
Enemy  HP: 110

=== PLAYER TURN ===
YOU used: ATTACK
Player HP: 55
Enemy  HP: 100

=== ENEMY TURN ===
ENEMY used: ATTACK
Player HP: 30
Enemy  HP: 100

=== PLAYER TURN ===
YOU used: ATTACK
Player HP: 30
Enemy  HP: 90

=== ENEMY TURN ===
ENEMY used: ATTACK
Player HP: 5
Enemy  HP: 90

=== PLAYER TURN ===
YOU used: ATTACK
Player HP: 5
Enemy  HP: 80

=== ENEMY TURN ===
ENEMY used: ATTACK
Player HP: 0
Enemy  HP: 80

You lose.
=== BATTLE FINISHED ===
